In [2]:
import torch 
import torchvision.datasets as dsets 
import torchvision.transforms as transforms 
import torch.nn as nn 
import torch.nn.functional as F 
from torch.autograd import Variable 

In [3]:
torch.manual_seed(1)

In [4]:
word_to_ix = {"hello":0, "world":1}

In [5]:
embeds = nn.Embedding(2,5)
lookup_tensor = torch.tensor([word_to_ix["hello"]],dtype = torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward>)


In [9]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [21]:
trigrams = [(test_sentence[i], test_sentence[i+1], test_sentence[i+2]) for i in range(len(test_sentence) - 2)]
print(trigrams)

[('When', 'forty', 'winters'), ('forty', 'winters', 'shall'), ('winters', 'shall', 'besiege'), ('shall', 'besiege', 'thy'), ('besiege', 'thy', 'brow,'), ('thy', 'brow,', 'And'), ('brow,', 'And', 'dig'), ('And', 'dig', 'deep'), ('dig', 'deep', 'trenches'), ('deep', 'trenches', 'in'), ('trenches', 'in', 'thy'), ('in', 'thy', "beauty's"), ('thy', "beauty's", 'field,'), ("beauty's", 'field,', 'Thy'), ('field,', 'Thy', "youth's"), ('Thy', "youth's", 'proud'), ("youth's", 'proud', 'livery'), ('proud', 'livery', 'so'), ('livery', 'so', 'gazed'), ('so', 'gazed', 'on'), ('gazed', 'on', 'now,'), ('on', 'now,', 'Will'), ('now,', 'Will', 'be'), ('Will', 'be', 'a'), ('be', 'a', "totter'd"), ('a', "totter'd", 'weed'), ("totter'd", 'weed', 'of'), ('weed', 'of', 'small'), ('of', 'small', 'worth'), ('small', 'worth', 'held:'), ('worth', 'held:', 'Then'), ('held:', 'Then', 'being'), ('Then', 'being', 'asked,'), ('being', 'asked,', 'where'), ('asked,', 'where', 'all'), ('where', 'all', 'thy'), ('all', 't

In [12]:
vocab = set(test_sentence)

In [17]:
word_to_ix = {word : i for i,word in enumerate(vocab)}

In [18]:
class ngramlanguagemodeler(nn.Module):
    def __init__(self,vocab_size, embedding_dim, context_size):
        super(ngramlanguagemodeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size*embedding_dim , 128)
        self.linear2 = nn.Linear(128,vocab_size)
    def forward(self, x):
        embeds = self.embeddings(x).view(1,-1)
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        out = F.log_softmax(out, dim = 1)
        return out 

In [19]:
losses = []
criterion = nn.NLLLoss()
model = ngramlanguagemodeler(len(vocab), EMBEDDING_DIM, 2)

In [20]:
otimizer = torch.optim.SGD(model.parameters(), lr = 0.001)

In [ ]:
for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:
        cont = torch.tensor([word_to_ix[w] for w in context])
        